In [ ]:
import os, shutil
import pandas as pd
import numpy  as np
from tqdm import tqdm
from WCFAdox import PCAX
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.font_manager import FontProperties
from tabulate import tabulate

## Download Raw Data

In [ ]:
# Convert 'YYYYMMDD' string to 'YYYY-MM-DD'
def date_transform(text):
    return text[:4] + '-' + text[4:6] + '-' + text[6:]


# Clear and reconstruct folder for fresh data saving
def clear_construct_folder(foldername):
    try:
        shutil.rmtree(foldername)
    except:
        os.makedirs(foldername)
    else:
        os.makedirs(foldername)

In [ ]:
# Get data for a list of foreign broker names
def get_foreign_broker_data(foldername, startDate, endDate, stock, brokers):
    PX = PCAX("10.216.8.148")
    df_all = PX.Sil_Data("個股券商進出明細", "D", stock, startDate, endDate, isst="Y")
    df_all["日期"] = df_all["日期"].astype(str).apply(date_transform)
    df_all = df_all.iloc[::-1]

    combined = []

    for broker in brokers:
        df = df_all[df_all["券商名稱"] == broker].copy()
        if not df.empty:
            output_path = f"{foldername}\\{broker}_{stock}.xlsx"
            df.to_excel(output_path, index=False, encoding="utf-8", engine="xlsxwriter")
            combined.append(df)

    if combined:
        merged_df = pd.concat(combined)
        merged_df.to_excel(f"{foldername}\\foreign_{stock}.xlsx", index=False, encoding="utf-8", engine="xlsxwriter")

        
# Get stock trading data
def get_stock_data(foldername, startDate, endDate, stock):
    PX = PCAX("10.216.8.148")
    df = PX.Sil_Data("日盤中零股交易行情", "D", stock, startDate, endDate, isst="Y")
    df["日期"] = df["日期"].astype(str).apply(date_transform)
    df["日期"] = pd.to_datetime(df["日期"], format="%Y-%m-%d", yearfirst=True)
    df = df.sort_values(by="日期")
    df["日期"] = df["日期"].dt.strftime("%Y-%m-%d")

    path = f"{foldername}\\stock_{stock}.xlsx"
    with pd.ExcelWriter(path, engine='xlsxwriter', datetime_format='YYYY-MM-DD', date_format='YYYY-MM-DD') as writer:
        df.to_excel(writer, index=False)

        
# Get TWII and futures data
def get_TWII_data(foldername, startDate, endDate):
    PX = PCAX("10.216.8.148")

    df = PX.Sil_Data("重要國際指數", "D", "#TWII", startDate, endDate, isst="N")
    df["日期"] = df["日期"].astype(str).apply(date_transform)
    df["日期"] = pd.to_datetime(df["日期"], format="%Y-%m-%d", yearfirst=True)
    df = df.sort_values(by="日期")
    df["日期"] = df["日期"].dt.strftime("%Y-%m-%d")

    df = df[["日期", "開盤價", "最高價", "最低價", "收盤價", "漲跌", "漲跌幅(%)", "成交量"]]
    df.columns = ["date", "TWII_opening", "TWII_highest", "TWII_lowest", "TWII_closing",
                  "TWII_priceDiff", "TWII_diffPercent", "TWII_volume"]

    path = f"{foldername}\\TWII.xlsx"
    with pd.ExcelWriter(path, engine='xlsxwriter', datetime_format='YYYY-MM-DD', date_format='YYYY-MM-DD') as writer:
        df.to_excel(writer, index=False)

    df = PX.Sil_Data("期貨交易行情表", "D", "TX", startDate, endDate, isst="N")
    df["日期"] = df["日期"].astype(str).apply(date_transform)
    df["日期"] = pd.to_datetime(df["日期"], format="%Y-%m-%d", yearfirst=True)
    df = df.sort_values(by="日期")
    df["日期"] = df["日期"].dt.strftime("%Y-%m-%d")

    df = df[["日期", "開盤價", "最高價", "最低價", "收盤價", "漲跌", "漲幅(%)", "成交量"]]
    df.columns = ["date", "future_opening", "future_highest", "future_lowest", "future_closing",
                  "future_priceDiff", "future_diffPercent", "future_volume"]

    path = f"{foldername}\\TWII_future.xlsx"
    with pd.ExcelWriter(path, engine='xlsxwriter', datetime_format='YYYY-MM-DD', date_format='YYYY-MM-DD') as writer:
        df.to_excel(writer, index=False)

        
# Main function to get all raw data
def get_raw_data(stock, dataFolder, startDate, endDate, brokers):
    foldername = f"{dataFolder}\\raw data"
    clear_construct_folder(foldername)

    get_foreign_broker_data(foldername, startDate, endDate, stock, brokers)
    get_stock_data(foldername, startDate, endDate, stock)
    get_TWII_data(foldername, startDate, endDate)

## Create Dataset

In [ ]:
def data_selection(dataFolder, stock, brokers):
    foldername = f"{dataFolder}\\raw data"
    path1 = f"{foldername}\\stock_{stock}.xlsx"
    path2 = f"{foldername}\\foreign_{stock}.xlsx"
    df1 = pd.read_excel(path1, engine="openpyxl")
    df2 = pd.read_excel(path2, engine="openpyxl")

    selected_dfs = []
    
    for broker in brokers:
        broker_df = df2[df2["券商"] == broker].copy()
        df = pd.merge(broker_df, df1, on=["日期"], how='outer')
        df = df[["日期", "開盤價", "最高價", "最低價", "收盤價", "漲跌", "漲幅(%)", "成交量(股)", "張增減", "買張", "賣張"]]
        df.columns = ["date", "opening", "highest", "lowest", "closing", "priceDiff", "diffPercent", "volume", "increment", "buy", "sell"]
        df = df.sort_values(by="date")
        df.to_excel(f"{foldername}\\{stock}_{broker}_selection.xlsx", index=False, encoding="utf-8", engine="xlsxwriter")
        selected_dfs.append(df)

    return selected_dfs

In [ ]:
def data_processing(df, dataFolder, stock, brokers):
    
    for broker in brokers:
        output_name = f"{stock}_{broker}"
        df["excessBuy"]   = (df["increment"].astype(float) >= 0).astype(int)
        df["marketShare"] = (df["buy"] + df["sell"]) / df["volume"]
        df["diffPercent"] = df["diffPercent"].astype(float)
        df["days"]        = 0
        df["acmlPercent"] = 0
        acml = 0

        for ind in range(1, len(df)):
            diff = df.iloc[ind]["diffPercent"]
            prev_day = df.iloc[ind - 1]["days"]
            if diff == 0:
                df.at[ind, "days"] = prev_day
                df.at[ind, "acmlPercent"] = acml
            elif diff > 0:
                df.at[ind, "days"] = prev_day + 1 if prev_day >= 0 else 1
                acml = diff if prev_day < 0 else acml + diff
            else:
                df.at[ind, "days"] = prev_day - 1 if prev_day <= 0 else -1
                acml = diff if prev_day > 0 else acml + diff
            df.at[ind, "acmlPercent"] = acml

        df["daysSoFar"] = df["days"].shift(1)
        df["change"] = (df["closing"] / df.iloc[0]["closing"]) - 1
        df["change1"] = df["change"].shift(1)

        for i in [1, 2, 3]:
            df[f"increment{i}"] = df["increment"].shift(i)
        df["totalExcess1"] = df["increment1"]
        df["totalExcess2"] = df["increment1"] + df["increment2"]
        df["totalExcess3"] = df["increment1"] + df["increment2"] + df["increment3"]

        buy_th  = df[df["increment"] > 0]["increment"].quantile(0.8)
        sell_th = df[df["increment"] < 0]["increment"].quantile(0.2)

        df["extreme"] = 0
        df["BUY"]  = (df["increment"] > buy_th).astype(int)
        df["SELL"] = (df["increment"] < sell_th).astype(int)
        df.loc[df["BUY"]  == 1, "extreme"] = 1
        df.loc[df["SELL"] == 1, "extreme"] = 2

        df["positive3"] = (df["days"] >= 3).astype(int)
        df["negative3"] = (df["days"] <= -3).astype(int)
        df["positive2"] = (df["days"].shift(1) >= 2).astype(int)
        df["negative2"] = (df["days"].shift(1) <= -2).astype(int)

        for i in range(1, 4):
            df[f"buy{i}"]  = df["buy"].shift(i, fill_value=0)
            df[f"sell{i}"] = df["sell"].shift(i, fill_value=0)

        df["increaseWithin"] = (df["highest"] / df["opening"]) - 1
        df["decreaseWithin"] = (df["lowest"] / df["opening"]) - 1

        for i in range(1, 4):
            df[f"closing{i}"] = df["closing"].shift(i)
            df[f"diffPcnt{i}"] = (df["opening"] / df[f"closing{i}"]) - 1
            df[f"diff{i}"] = df["opening"] - df[f"closing{i}"]

        df["incrementDiff"] = df["increment"] - df["increment"].shift(1)
        df.to_excel(f"{dataFolder}\\{output_name}_preprocessed.xlsx", index=False, encoding="utf-8", engine="xlsxwriter")


## Example

In [ ]:
dataFolder = "C:\\Users\\user.Y220026097\\Desktop\\UBS"
startDate  = "20211027"
endDate    = "20221031"

stocks     = ["2388", "2498"]
brokers    = ["新加坡商瑞銀", "高盛證券", "摩根大通", "摩根士丹利"]

In [ ]:
for stock in stocks:
    get_raw_data(dataFolder, startDate, endDate, stock, brokers)
    data_selection (dataFolder, stock, brokers)
    data_processing(dataFolder, stock, brokers)